In [42]:
#--------------------------------------------------------------------------LIBRARIES---------------------------------------------------------------------------------#
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from scipy.stats import linregress
import yfinance as yf

In [43]:
#--------------------------------------------------------------------------INPUTS---------------------------------------------------------------------------------#

ticker = input("Enter your ticker")
ticker_yf = str(ticker + ".NS")

log = input("A - for 5yr-exit and B - for 10yr-exit: ").strip()
if log == 'a' or log == 'A': years = 5
elif log == 'b' or log == 'B': years = 10
else: years = "Invalid input"

print(f'ticker : {ticker}')

ticker : TCS


In [44]:
#--------------------------------------------------------------------------DATA SCRAPPING----------------------------------------------------------------------------#
url = f'https://www.screener.in/company/{ticker}/consolidated/'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0 Safari/537.36"
}
response = requests.get(url, headers=headers)


########################################################################################################################################################################
#--------------------------------------------------------------------------MARKET DATA---------------------------------------------------------------------------------#
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extracting ratios
    ratios_section = soup.find('ul', id='top-ratios')
    market_cap = current_price = returns = None

    for item in ratios_section.find_all('li'):
        name = item.find('span', class_='name').get_text(strip=True)
        value = ''.join(item.find('span', class_='value').stripped_strings)

        # Extract only numbers using regex
        number_value = re.sub(r'[^\d.]', '', value)  # Allow decimals

        # print(value)
        if name == "Market Cap":
            market_cap = float(number_value)
        elif name == "Current Price":
            current_price = float(number_value)
        elif name == "ROE":
            returns = float(number_value)/100  # Convert to float

    # Calculating total shares
    if market_cap and current_price:
        total_shares = market_cap / current_price
    else:
        total_shares = "N/A"
else:
    print("Failed to retrieve data, Status Code:", response.status_code)

########################################################################################################################################################################
#------------------------------------------------------------------------CAGR RETURNS----------------------------------------------------------------------------------#

# Check response status
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Initialize global variables
    returns_10yr = returns_5yr = None

    # Locate the table with "Return on Equity"
    tables = soup.find_all("table", class_="ranges-table")
    
    for table in tables:
        header = table.find("th").get_text(strip=True)
        
        # Check if it's the "Return on Equity" table
        if "Stock Price CAGR" in header:
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                if cells:
                    period = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True).replace("%", "")  # Remove % symbol
                    value = float(value)  # Convert to float

                    # Store values based on period
                    if period == "10 Years:":
                        returns_10yr = value/100
                    elif period == "5 Years:":
                        returns_5yr = value/100

    if years == 5 :
        if returns > returns_5yr :  returns = returns_5yr
    elif years == 10 :
        if returns > returns_10yr : returns = returns_10yr

else:
    print("Failed to retrieve data, Status Code:", response.status_code)

########################################################################################################################################################################
#------------------------------------------------------------------------PROFIT GROWTH----------------------------------------------------------------------------------#

HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    
    }

# Send request
response = requests.get(url, headers=headers)

# Check response status
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Initialize global variables
    profit_10yr = profit_5yr = None

    # Locate the table with "Return on Equity"
    tables = soup.find_all("table", class_="ranges-table")
    
    for table in tables:
        header = table.find("th").get_text(strip=True)
        
        # Check if it's the "Return on Equity" table
        if "Compounded Profit Growth" in header:
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                if cells:
                    period = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True).replace("%", "")  # Remove % symbol
                    value = float(value)  # Convert to float

                    # Store values based on period
                    if period == "10 Years:":
                        profit_10yr = value/100
                    elif period == "5 Years:":
                        profit_5yr = value/100
    if years == 5 : 
        profit = profit_5yr
        # print(profit)
    elif years == 10 : profit = profit_10yr
else:
    print("Failed to retrieve data, Status Code:", response.status_code)

########################################################################################################################################################################
#------------------------------------------------------------------------FREE CASH GROWTH----------------------------------------------------------------------------------#

############### SCRAP CASH FLOW
def scrape_company_data(ticker, section_ids):
    url = f"https://www.screener.in/company/{ticker}/consolidated/#quarters"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Dictionary to store filtered dataframes for each section
    section_data = {}

    for section_id in section_ids:
        # Locate the section table
        section = soup.find("section", {"id": section_id})
        table = section.find("table", {"class": "data-table"}) if section else None
        
        if table:
            # Extract headers
            headers = [th.text.strip() for th in table.find("thead").find_all("th")]

            # Extract data
            data = []
            for row in table.find("tbody").find_all("tr"):
                cells = [cell.text.strip() for cell in row.find_all("td")]
                if len(cells) > 1:
                    data.append(cells)

            # Create DataFrame if data is available
            if data:
                df = pd.DataFrame(data, columns=headers)
                section_data[section_id] = df
        else:
            print(f"⚠️ No table found for section ID: {section_id}")

    return section_data

# Example Usage
# section_ids = ["profit-loss", "cash-flow", "balance-sheet"]
section_ids = ["cash-flow"]
dataframes = scrape_company_data(ticker, section_ids)

# ✅ Extract CFO (Last 5 Years)
df_cash_flow = dataframes.get("cash-flow")
cfo_row = df_cash_flow[df_cash_flow.iloc[:, 0].str.contains("Cash from Operating Activity", case=False, na=False)]

if not cfo_row.empty:
    cfo_values = cfo_row.iloc[0, 1:].apply(lambda x: float(x.replace(",", "")) if x else None).tolist()

    if pd.isna(cfo_values[-1]): 
        cfo_last_5_years = cfo_values[-6:-1]  # Gets the 5th to last up to but not including the last
    else : 
        cfo_last_5_years = cfo_values[-5:]  # last 5 values
else:
    print("⚠️ Could not find 'Cash from Operating Activity' row.")

############### INTERNAL TICKER
def get_internal_ticker_id(stock_symbol, headers):
    """Fetch the internal ticker ID from Screener"""
    company_url = f"https://www.screener.in/company/{stock_symbol}/consolidated/"
    response = requests.get(company_url, headers=headers)

    if response.ok:
        soup = BeautifulSoup(response.content, "html.parser")
        company_info_div = soup.find("div", {"id": "company-info"})
        
        # Extract company ID
        if company_info_div:
            ticker_id = company_info_div.get("data-company-id")
            if ticker_id:
                # print(ticker_id)
                return ticker_id

    print("Failed to retrieve ticker ID")
    return None

############### SCRAP CAPEX
def fetch_cash_flow_data(stock_symbol, headers):
    """Fetch last 5 values of 'Fixed assets purchased' from Screener"""
    ticker_id = get_internal_ticker_id(stock_symbol, headers)
    
    if ticker_id:
        api_url = f"https://www.screener.in/api/company/{ticker_id}/schedules/?parent=Cash+from+Investing+Activity&section=cash-flow&consolidated="
        response = requests.get(api_url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            fixed_assets_data = data.get('Fixed assets purchased', {})

            if fixed_assets_data:
                sorted_years = sorted(fixed_assets_data.keys())[-5:]
                last_5_values = []

                for year in sorted_years:
                    raw_value = fixed_assets_data[year].replace(",", "")
                    try:
                        value = float(raw_value)
                        last_5_values.append(value)
                    except ValueError:
                        last_5_values.append(None)

                
                return last_5_values

    print("Failed to fetch " \
    " data.")
    return []

# Example Usage
capex_array = fetch_cash_flow_data(ticker, HEADERS)

freecash = []
for i in range (0,5):
    val = cfo_last_5_years[i] - capex_array[i]
    freecash.append(val)


def adjusted_cagr(values):
    # Find the first positive value as new starting point
    for i in range(len(values) - 1):
        if values[i] > 0:
            initial = values[i]
            final = values[-1]
            years = len(values) - 1 - i
            break
    else:
        return None  # All values are non-positive; CAGR not possible

    cagr = (final / initial) ** (1 / years) - 1
    return cagr * 100

cagr_result = adjusted_cagr(freecash)/100

########################################################################################################################################################################
#------------------------------------------------------------------------EBITDA DATA----------------------------------------------------------------------------------#
def scrape_company_data(ticker, section_ids):
    url = f"https://www.screener.in/company/{ticker}/consolidated/#quarters"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Dictionary to store filtered dataframes for each section
    section_data = {}

    # Array to store only Operating Profit values
    ebitda = []

    for section_id in section_ids:
        # Locate the section table
        section = soup.find("section", {"id": section_id})
        table = section.find("table", {"class": "data-table"}) if section else None
        
        if table:
            # Extract headers
            headers = [th.text.strip() for th in table.find("thead").find_all("th")]

            # Extract data
            data = []
            for row in table.find("tbody").find_all("tr"):
                cells = [cell.text.strip() for cell in row.find_all("td")]

                # Handling based on section ID
                if section_id == "profit-loss" and len(cells) >= 2:
                    # Store only the "Operating Profit" row
                    if "Operating Profit" in cells[0]:
                        data.append(["Operating Profit"] + cells[1:])
                    elif "Financing Profit" in cells[0]:
                        data.append(["Operating Profit"] + cells[1:])

            # Create DataFrame if data is available
            if data:
                # Use the first row as the column names (years)
                column_names = headers[1:]  # All except the first column (which is the metric name)
                df = pd.DataFrame(data, columns=["Metric"] + column_names)
                # df = df.iloc[:, :-1]  # This slices the DataFrame to exclude the last column - TTM
                section_data["Operating Profit"] = df
                ebitda = [
                    float(val.replace(',', '')) for val in df.iloc[0, 1:].tolist()  # Convert to float and store as list
                ]
        else:
            print(f"⚠️ No table found for section ID: {section_id}")

    return section_data, ebitda

# Example Usage
section_ids = ["profit-loss", "cash-flow", "balance-sheet"]
dataframes, ebitda = scrape_company_data(ticker, section_ids)

########################################################################################################################################################################
#----------------------------------------------------------------------FINANCIAL DATA----------------------------------------------------------------------------------#

############################### SCRAPING OF DATA
def scrape_company_data(ticker, section_ids):
    url = f"https://www.screener.in/company/{ticker}/consolidated/#quarters"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Dictionary to store filtered dataframes for each section
    section_data = {}

    for section_id in section_ids:
        # Locate the section table
        section = soup.find("section", {"id": section_id})
        table = section.find("table", {"class": "data-table"}) if section else None
        
        if table:
            # Extract headers
            headers = [th.text.strip() for th in table.find("thead").find_all("th")]

            # Extract data
            data = []
            for row in table.find("tbody").find_all("tr"):
                cells = [cell.text.strip() for cell in row.find_all("td")]

                # Handling based on section ID
                if section_id == "profit-loss" and len(cells) >= 2:
                    # Second-last column for profit-loss
                    data.append([cells[0], cells[-1]])
                elif section_id == "cash-flow" and len(cells) >= 1:
                    # Last column for cash-flow
                    data.append([cells[0], cells[-1]])
                elif section_id == "balance-sheet" and len(cells) >= 2:
                    # Second-last column for balance-sheet
                    data.append([cells[0], cells[-1]])

            # Create DataFrame if data is available
            if data:
                column_name = headers[-2] if section_id in ["profit-loss", "balance-sheet"] else headers[-1]
                df = pd.DataFrame(data, columns=[headers[0], column_name])
                section_data[section_id] = df
        else:
            print(f"⚠️ No table found for section ID: {section_id}")
    # print(data)
    return section_data

############################### FILTERING DATA
def extract_variables(dataframes):
    global cash_from_operating_activity, equity_capital, reserves, borrowings, interest, tax_percent, dpr,tax_expense,debt,tax
    # Extract from cash-flow
    if "cash-flow" in dataframes:
        df_cash_flow = dataframes["cash-flow"]
        cash_from_operating_activity = df_cash_flow.loc[df_cash_flow.iloc[:, 0].str.contains("Cash from Operating Activity", case=False, na=False), df_cash_flow.columns[-1]].values
        cash_from_operating_activity = float(cash_from_operating_activity[0].replace(',', '').strip()) if len(cash_from_operating_activity) > 0 else None

    # Extract from balance-sheet
    if "balance-sheet" in dataframes:
        df_balance_sheet = dataframes["balance-sheet"]
        equity_capital = df_balance_sheet.loc[df_balance_sheet.iloc[:, 0].str.contains("Equity Capital", case=False, na=False), df_balance_sheet.columns[-1]].values
        equity_capital = float(equity_capital[0].replace(',', '').strip()) if len(equity_capital) > 0 else None
        
        reserves = df_balance_sheet.loc[df_balance_sheet.iloc[:, 0].str.contains("Reserves", case=False, na=False), df_balance_sheet.columns[-1]].values
        reserves = float(reserves[0].replace(',', '').strip()) if len(reserves) > 0 else None
        
        borrowings = df_balance_sheet.loc[df_balance_sheet.iloc[:, 0].str.contains("Borrowing", case=False, na=False), df_balance_sheet.columns[-1]].values
        borrowings = float(borrowings[0].replace(',', '').strip()) if len(borrowings) > 0 else None
        debt = borrowings 

    # Extract from profit-loss
    if "profit-loss" in dataframes:
        df_profit_loss = dataframes["profit-loss"]
        interest = df_profit_loss.loc[df_profit_loss.iloc[:, 0].str.contains("Interest", case=False, na=False), df_profit_loss.columns[-1]].values
        tax_percent = df_profit_loss.loc[df_profit_loss.iloc[:, 0].str.contains("Tax %", case=False, na=False), df_profit_loss.columns[-1]].values
        pat = df_profit_loss.loc[df_profit_loss.iloc[:, 0].str.contains("Net Profit", case=False, na=False), df_profit_loss.columns[-1]].values
        pbt = df_profit_loss.loc[df_profit_loss.iloc[:, 0].str.contains("Profit before tax", case=False, na=False), df_profit_loss.columns[-1]].values
        dpr = df_profit_loss.loc[df_profit_loss.iloc[:, 0].str.contains("Dividend Payout %", case=False, na=False), df_profit_loss.columns[-1]].values


        interest = float(interest[0].replace(',', '').strip()) if len(interest) > 0 else None
        tax_percent = tax_percent[0] if len(tax_percent) > 0 else None
        tax_percent = float(tax_percent.replace('%', '').strip())/100
        tax = tax_percent

        dpr = dpr[0] if len(dpr) > 0 else None
        dpr = float(dpr.replace('%', '').strip())/100

        pbt = pbt[0] 
        pbt = float(pbt.replace(',', '').strip())

        pat = pat[0]
        pat = float(pat.replace(',', '').strip())
        tax_expense = pbt-pat

section_ids = ["profit-loss", "cash-flow", "balance-sheet"]
dataframes = scrape_company_data(ticker, section_ids)
extract_variables(dataframes)

########################################################################################################################################################################
#----------------------------------------------------------------------SOME RATIOS----------------------------------------------------------------------------------#
capex = capex_array[-1]
CFO = cfo_last_5_years[-1]
FCFF = CFO - capex - interest*(1-tax)

In [55]:
#--------------------------------------------------------------------------WACC-----------------------------------------------------------------------------------#
####################################### To calculate Beta from Yahoo
t_debt = debt

def calculate_beta(ticker_yf, period="1y"):
    global benchmark, slope
    benchmark = "^NSEI"  # Default to SENSEX

    # Fetch stock and market data
    stock_data = yf.download(ticker_yf, period=period, interval="1d")
    market_data = yf.download(benchmark, period=period, interval="1d")

    # Check if stock data is empty
    if stock_data.empty:
        print(f"❌ Error: No data found for {ticker_yf}. Check the symbol or period.")
        return

    # Check if market data is empty and switch to NIFTY if needed
    if market_data.empty:
        print(f"⚠️ Warning: No data for SENSEX. Trying SENSEX instead...")
        benchmark = "^BSESN"
        market_data = yf.download(benchmark, period=period, interval="1d")

        if market_data.empty:
            print(f"❌ Error: No data for {benchmark} either. Exiting.")
            return

    # Select appropriate closing price columns
    stock_column = "Close" if "Close" in stock_data.columns else "Close"
    market_column = "Close" if "Close" in market_data.columns else "Close"

    # Extract price data safely
    stock = stock_data[stock_column]
    market = market_data[market_column]

    # Ensure both stock and market data are valid pandas Series
    if stock.empty or market.empty:
        print("❌ Error: Stock or market data is empty. Cannot compute beta.")
        return

    # Merge stock and market data on common dates
    data = pd.concat([stock, market], axis=1, keys=["Stock", "Market"]).dropna()

    # Ensure there are enough data points
    if data.empty:
        print("❌ Error: Not enough data to proceed with beta calculation.")
        return

    # Compute daily percentage returns
    data['Stock Returns'] = data['Stock'].pct_change()
    data['Market Returns'] = data['Market'].pct_change()
    data.dropna(inplace=True)  # Ensure no NaNs before regression

    # Check if there are enough return data points
    if data.empty or len(data) < 2:
        print("❌ Error: Not enough return data to calculate Beta.")
        return

    # Perform linear regression to estimate Beta
    slope, _, r_value, _, _ = linregress(data['Market Returns'], data['Stock Returns'])

    # print(f"\nBeta for {ticker_yf} (Benchmark: {benchmark}) over {period}: {slope:.2f}")
    return slope

beta = calculate_beta(ticker_yf)

############################################################################################################################################
#------------------------------------------------------------MARKET RETURNS-----------------------------------------------------------------#

url = f'https://www.screener.in/company/NIFTY/'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extracting ratios
    ratios_section = soup.find('ul', id='top-ratios')
    cagr_5 = cagr_10 = None

    for item in ratios_section.find_all('li'):
        name = item.find('span', class_='name').get_text(strip=True)
        cash = ''.join(item.find('span', class_='value').stripped_strings)

        # Extract only numbers using regex
        number_value = re.sub(r'[^\d.]', '', cash)  # Remove all except digits and decimal

        # Corrected condition to check the name
        if name == "CAGR 5Yr":
            cagr_5 = float(number_value)/100  # Convert to float
        elif name == "CAGR 10Yr":
            cagr_10 = float(number_value)/100

else:
    print("Failed to retrieve data, Status Code:", response.status_code)
if years == 5 : Rm = cagr_5
elif years == 10 : Rm = cagr_10

############################################################################################################################################
#-----------------------------------------------------COST OF EQUITY AND COST OF DEBT------------------------------------------------------#

####### Cost Of Equity
rfr = 6.44/100                                                             # 10-yr G-sec yields
# Rm = 15/100                                                             #Hardcoded 15% as per the given website (there is a code to find cagr of 5 and 10 years)

#Calculation
C_equity = (rfr + beta*(Rm - rfr))

####### Cost Of Debt
C_debt = interest/t_debt

############################################################################################################################################
#-----------------------------------------------------COST OF EQUITY AND COST OF DEBT------------------------------------------------------#

####### WACC
t_equity = market_cap                                                       #Market value of equity
wacc = (t_debt*C_debt*(1-tax) + (t_equity*C_equity))/(t_debt+t_equity)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [54]:
#---------------------------------------------------------------------DCF CALCULATION---------------------------------------------------------------------#
long_growth_rate = 0.03    # Mostly aligned with country's inflation or real gdp

############### DCF CALCULATION
terminal = long_growth_rate
discount = wacc
FCF = FCFF

# Compute terminal value
def calculate_terminal_value(fcf_initial, growth_rate, long_growth_rate, wacc, years):
    # Calculate FCFs for the last 3 years
    fcf_years = [fcf_initial * ((1 + growth_rate) ** (years - i)) for i in range(3)]
    avg_fcf_last_3_years = sum(fcf_years) / 3

    # Calculate terminal value using the average
    terminal_value = (avg_fcf_last_3_years * (1 + long_growth_rate)) / (wacc - long_growth_rate)
    return terminal_value


terminal_value1 = calculate_terminal_value(FCF, profit, terminal, discount, years)
terminal_value2 = calculate_terminal_value(FCF, cagr_result, terminal, discount, years)

def discounted_cash_flow(fcf_initial, growth_rate, wacc, years, terminal_value):
    global fcf_list
    """
    Calculate the discounted cash flow (DCF) valuation.
    
    Returns:
    - Present value of the firm
    """
    fcf_list = [fcf_initial * ((1 + growth_rate) ** t) for t in range(1, years + 1)]
    
    # DCF value for forecasted years
    # dcf_value = sum(fcf / ((1 + wacc) ** (t + 1)) for t, fcf in enumerate(fcf_list))
    dcf_value = 0
    for t, fcf in enumerate(fcf_list):
        discounted_fcf = fcf / ((1 + wacc) ** (t + 1))
        dcf_value += discounted_fcf
        # print(f"Year {t+1}: FCF = {fcf:.2f}, Discounted FCF = {discounted_fcf:.2f}")

    
    # Discount terminal value
    discounted_terminal_value = terminal_value / ((1 + wacc) ** years)
    
    return dcf_value + discounted_terminal_value

# Example usage:
dcf_valuation1 = discounted_cash_flow(FCF, profit, discount, years, terminal_value1)
dcf_valuation2 = discounted_cash_flow(FCF, cagr_result, discount, years, terminal_value2)
equity1 = dcf_valuation1                   
equity2 = dcf_valuation2 


# Calculate fair prices using two different growth rates
fair_price1 = equity1 / total_shares if total_shares else 0
fair_price2 = equity2 / total_shares if total_shares else 0

# Ensure lower price is shown first
low_fair, high_fair = sorted([fair_price1, fair_price2])

# Calculate safe prices with margin of safety.
safe = 0/100
safe_price1 = low_fair * (1 - safe)
safe_price2 = high_fair * (1 - safe)



In [51]:
#---------------------------------------------------------------------DCF CALCULATION PRINT---------------------------------------------------------------------#
#DCF Calculation
print(f'Free Cash Flow Rate(%) : {cagr_result*100:.2f}') 
print(f'Profit Growth(%) : {profit*100:.2f}')
print(f'Terminal growth rate(%) : {long_growth_rate*100}')
print(f'Discount Rate(%) : {wacc*100:.2f}')
print("-"*50)
print(f'Projected FCF : {fcf_list}')
print(f"\n🎯 Fair Price Range: ₹{low_fair:.2f} - ₹{high_fair:.2f}")
print(f"🛡️ Safe Price Range (with {int(safe * 100)}% margin of safety): ₹{safe_price1:.2f} - ₹{safe_price2:.2f}")

Free Cash Flow Rate(%) : 5.92
Profit Growth(%) : 10.00
Terminal growth rate(%) : 3.0
Discount Rate(%) : 8.67
--------------------------------------------------
Projected FCF : [55343.32024924346, 58622.016081195856, 62094.9511873741, 65773.63285530517, 69670.25009698584, 73797.71403618075, 78169.70068837928, 82800.6962751585, 87706.04522309823, 92902.00100628837]

🎯 Fair Price Range: ₹3175.85 - ₹4244.24
🛡️ Safe Price Range (with 0% margin of safety): ₹3175.85 - ₹4244.24


In [ ]:
##################### PRINT

# Print stored values
print(f'Ticker : {ticker}')
print("Market Cap:", market_cap)
print("Current Price:", current_price)
print("Total number of shares:", total_shares)
print("Current ROE (%):", round(returns*100,2))
print("-"*50)

# Display the extracted returns
print("Retun 5 Years(%):", returns_5yr*100)
print("Retun 10 Years(%):", returns_10yr*100)
print("Return(%):", returns*100)
print("-"*50)

# Display the extracted profit
print("Profit-growth 5 Years(%):", profit_5yr*100)
print("Profit-growth 10 Years(%):", profit_10yr*100)
print("Profit-growth for calculation(%):", profit*100)
print("-"*50)

# Free Cash Flow Values
print(f"CFO (Last 5 Years): {cfo_last_5_years}")
print("CAPEX array (last 5 years):", capex_array)
print(f'Last 5 years FCF : {freecash}')
print(f"Free cash flow CAGR: {cagr_result*100:.2f}%")
print("-"*50)

# Display the array of Operating Profit values
print(f"--- EBITDA Values Array ---")
print(ebitda)
print("-"*50)

# FINANCIAL STATEMENT DATA
print(f"Cash from Operating Activity: {cash_from_operating_activity}")
print(f"Equity Capital: {equity_capital}")
print(f"Reserves: {reserves}")
print(f"Debt: {debt}")
print(f"Interest: {interest}")
print(f"Tax %: {tax*100}")
print(f"dpr %: {dpr*100:.2f}")
print(f'Tax expense: {tax_expense:.2f}')
print("-"*50)

# Print Nifty Values
print(f"Beta for {ticker_yf} (Benchmark: {benchmark}) over {period}: {slope:.2f}")
print(f'5yr CAGR of NIFTY50(%) : {cagr_5*100:.2f}')
print(f'10yr CAGR of NIFTY50(%) : {cagr_10*100:.2f}')
print(f'Market return for {years}yr : {Rm*100}%')
print("-"*50)

#Cost of Equity and Cost of Debt
print(f'rfr : {rfr*100:.2f}%\nRm : {Rm*100:.2f}%\nBeta : {beta:.2f}')
print(f'CAPM = {rfr*100:.2f}% - {beta:.2f}*({Rm*100:.2f}%-{rfr*100:.2f}%)')
print(f'Cost of Equity(%) : {C_equity*100:.2f}%')
print(f'Cost of debt(%) : {C_debt*100:.2f}')
print("-"*50)

#WACC rate
print(f'Total Debt : {t_debt}')
print(f'Cost of debt(%) : {C_debt*100:.2f}')
print(f'Total Equity : {t_equity}')
print(f'Cost of Equity(%) : {C_equity*100:.2f}')
print(f'wacc = ( ({t_debt:.2f}*{C_debt*100:.2f}%*(1-{tax*100}%)) + ({t_equity:.2f}*{C_equity*100:.2f}%))/({t_debt:.2f}+{t_equity:.2f})')
print(f'wacc(%) : {wacc*100:.2f}')
print("-"*50)

# SOME RATIOS
print("CFO latest:", CFO)
print("CAPEX latest:", capex)
print("Free Cash Flow latest:", FCFF)
print("-"*50)

Ticker : TCS
Market Cap: 1169185.0
Current Price: 3232.0
Total number of shares: 361.75278465346537
Current ROE (%): 10.0
--------------------------------------------------
Retun 5 Years(%): 13.0
Retun 10 Years(%): 10.0
Return(%): 10.0
--------------------------------------------------
Profit-growth 5 Years(%): 9.0
Profit-growth 10 Years(%): 10.0
Profit-growth for calculation(%): 10.0
--------------------------------------------------
CFO (Last 5 Years): [38802.0, 39949.0, 41965.0, 44338.0, 48908.0]
CAPEX array (last 5 years): [-3176.0, -2995.0, -3100.0, -2674.0, -3937.0]
Last 5 years FCF : [41978.0, 42944.0, 45065.0, 47012.0, 52845.0]
Free cash flow CAGR: 5.92%
--------------------------------------------------
--- EBITDA Values Array ---
[25153.0, 24482.0, 30677.0, 32311.0, 32516.0, 39506.0, 42109.0, 46546.0, 53057.0, 59259.0, 64296.0, 67407.0]
--------------------------------------------------
Cash from Operating Activity: 48908.0
Equity Capital: 362.0
Reserves: 94394.0
Debt: 9392.0